In [5]:
# Import Libraries to get started

from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
import statsmodels.formula.api as sm
import matplotlib.pylab as plt

import dmba
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

%matplotlib inline

In [6]:
# Import each dataset seperately, assign to dataframes

df_customers = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
df_engagement = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
df_marketing = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')
df_transactions = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')

In [13]:
# Create main df by merging all datasets

df = pd.merge(df_customers,df_engagement, on='customer_id', how='outer')

In [15]:
df = pd.merge(df,df_marketing,on='customer_id', how='outer')

In [17]:
df = pd.merge(df,df_transactions,on='customer_id', how='outer')

In [18]:
df.columns

Index(['customer_id', 'join_date', 'last_purchase_date', 'age', 'gender',
       'location', 'number_of_site_visits', 'number_of_emails_opened',
       'number_of_clicks', 'campaign_id', 'response', 'promotion_type',
       'campaign_date', 'transaction_id', 'transaction_date',
       'transaction_amount', 'product_category'],
      dtype='object')